# PINECONEを用いたvectorサーチのデモ

llama_indexのバージョンが上がったので、それに伴いコードを修正

2. 技術ブログの登録

- https://gpt-index.readthedocs.io/en/stable/how_to/integrations/vector_stores.html


# 方針

- LangChain.Agentを利用せずに、LLamaIndex単体で実装

In [8]:
import os

from dotenv import load_dotenv
from llama_index.vector_stores import PineconeVectorStore
from llama_index import SimpleWebPageReader, LLMPredictor, ServiceContext, OpenAIEmbedding, GPTVectorStoreIndex, StorageContext
from llama_index.prompts.prompts import QuestionAnswerPrompt, RefinePrompt
import pinecone
from langchain.chat_models import ChatOpenAI

INDEX_NAME = "chatgpt-search-index"

PREDICTOR_MODEL_NAME = "gpt-3.5-turbo"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

INITIAL_URLS = [
    "https://dev.classmethod.jp/articles/lang-chain-agent-customized-by-llama-index-tool/",
    "https://yukoishizaki.hatenablog.com/entry/2020/05/24/145155",
    "https://runble1.com/gcp-terraform-cloud-run/"
]
ADDITIONAL_URL = "https://zenn.dev/tfutada/articles/acf8adbb2ba5be"

# カスタムテンプレートの作成
CUSTOM_TEXT_QA_PROMPT_TMPL = (
    "コンテキストは以下です. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "コンテキストが与えられた場合, "
    "質問に回答してください: {query_str}\n"
)
CUSTOM_TEXT_QA_PROMPT = QuestionAnswerPrompt(CUSTOM_TEXT_QA_PROMPT_TMPL)

CUSTOM_REFINE_PROMPT_TMPL = (
    "元の質問: {query_str}\n"
    "オリジナルの回答: {existing_answer}\n"
    "以下のコンテキストを使って、オリジナルの回答を推敲することができます.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "コンテキストを元に、オリジナルの回答を、より元の質問に沿ったものに推敲してください. "
    "もしコンテキストが有用なものでなければ、オリジナルの回答を返却してください"
)
CUSTOM_REFINE_PROMPT = RefinePrompt(CUSTOM_REFINE_PROMPT_TMPL)

# 環境変数の読み込み
load_dotenv('../.env')

True

In [3]:
# デバッグする際に実行
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# 既存DBへの接続 & 検索

In [5]:
# clientの作成
pinecone.init(api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENVIRONMENT"])
pinecone_index = pinecone.Index(INDEX_NAME)

vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index
)

In [6]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 48}},
 'total_vector_count': 48}

In [12]:
# indexのセットアップ
storage_context = StorageContext.from_defaults(
    vector_store = vector_store
)

llm_predictor = LLMPredictor(
    llm=ChatOpenAI(
        temperature=0, model_name=PREDICTOR_MODEL_NAME, openai_api_key=os.environ["OPENAI_API_KEY"]
    )
)

embed_model = OpenAIEmbedding(
    model=EMBEDDING_MODEL_NAME
)

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embed_model
)

index = GPTVectorStoreIndex.from_documents(
    documents=[], service_context=service_context, storage_context=storage_context
)

Unknown max input size for gpt-3.5-turbo, using defaults.
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [13]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 48}},
 'total_vector_count': 48}

In [ ]:
query_engine = index.as_query_engine(
    text_qa_template=CUSTOM_TEXT_QA_PROMPT,
    refine_template=CUSTOM_REFINE_PROMPT,
    similarity_top_k=3
)

search_query = "calibrationってどんな技術だっけ？"

response = query_engine.query(
    search_query
)

In [16]:
print(f"response: {response.response}")

reffer_urls = set([source_node.extra_info["url"] for source_node in response.source_nodes])
for i, url in enumerate(reffer_urls):
    print(f"参照url{i+1}: {url}")

response: Calibrationは、モデルの確率予測の信頼性を高めるための技術で、モデルの出力値を各クラスに属する確率に近づけることを指します。具体的には、Sigmoid / Platt ScaleやIsotonic Regressionなどの手法を使って、モデルの出力値を正解ラベルのクラス分布に近づけます。最近のニューラルネットワークは自信過剰で、Calibrationが必要な場合もあります。また、MMMなどの統計分析だけでは過学習している可能性があるため、交差検証を行うことが重要です。A/Bテストを行うことで、交差検証を補完し、モデルの信頼性を高めることができます。
参照url1: https://tjo.hatenablog.com/entry/2023/04/26/191100
参照url2: https://yukoishizaki.hatenablog.com/entry/2020/05/24/145155


/Users/ikegamikenshin/Documents/study/chatgpt-practice/.venv/lib/python3.9/site-packages/llama_index/data_structs/node.py:181: UserWarning: .extra_info is deprecated, use .node.extra_info instead
  warnings.warn(".extra_info is deprecated, use .node.extra_info instead")


In [17]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 48}},
 'total_vector_count': 48}

# データのインサート

In [18]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 48}},
 'total_vector_count': 48}

In [ ]:
search_query = "DIDについて教えて"

response = query_engine.query(
    search_query
)

print(f"response: {response.response}")

reffer_urls = set([source_node.extra_info["url"] for source_node in response.source_nodes])
for i, url in enumerate(reffer_urls):
    print(f"参照url{i+1}: {url}")

In [22]:
# webページからdocumentクラスを作成
samples_urls = ["https://zenn.dev/s1ok69oo/articles/e786bd6ee2d1f0"]


documents = SimpleWebPageReader(html_to_text=True).load_data(samples_urls)

1

In [23]:
# extra_infoにurlを追加
for document, url in zip(documents, samples_urls):
    document.extra_info = {"url": url}

In [ ]:
for document in documents:
        index.insert(document)

In [25]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 62}},
 'total_vector_count': 62}

In [ ]:
search_query = "DIDについて教えて"

response = query_engine.query(
    search_query
)

print(f"response: {response.response}")

reffer_urls = set([source_node.extra_info["url"] for source_node in response.source_nodes])
for i, url in enumerate(reffer_urls):
    print(f"参照url{i+1}: {url}")

In [27]:
query_response = pinecone_index.query(
    top_k=10,
    include_metadata=True,
    # dummyのベクトル
    vector=[0.1] * 1536,
    filter={
         "extra_info_url": {"$in": ["https://zenn.dev/s1ok69oo/articles/e786bd6ee2d1f0"]}
    }
)

In [32]:
len(query_response["matches"])

10